In [ ]:
import os
import sys
sys.path.append("..")
from nbr.preparation import Preprocess, save_split, Corpus
from nbr.trainer import NBRTrainer
from nbr.model import RepurchaseModule, NBRKNN
import torch
import random
import numpy as np
import optuna
import warnings
warnings.filterwarnings("ignore")

# TaFeng

Fix seed:

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 5 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [ ]:
corpus_path = "./data/"
dataset_name = "ta_feng"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 32266, #items = 23812, #clicks = 817741 (#illegal records = 0)
After preprocessing: #users = 7358, #items = 11202, #clicks = 368951
Saving dataset in ./data//data_ta_feng/...


In [ ]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Train Repurchase Module with best hyperparams (see ./testing_repurchasemodule.ipynb):

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=20,
    topk=10,
    early_stop_num=3
)

train dataset preparing...


100%|██████████| 7358/7358 [00:15<00:00, 461.82it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:03<00:00, 2337.01it/s]


test dataset preparing...


100%|██████████| 7357/7357 [00:02<00:00, 2753.44it/s]


In [ ]:
slrc_best_params = {'batch_size': 256, 'lr': 0.00011201144001505824, 'l2_reg_coef': 0.00011498224071460201}

params = {
    "model": RepurchaseModule(
        item_num=corpus.n_items,
        avg_repeat_interval=corpus.total_avg_interval
    ),
    "batch_size": slrc_best_params["batch_size"],
    "lr": slrc_best_params["lr"],
    "l2_reg_coef": slrc_best_params["l2_reg_coef"]
}

trainer.init_hyperparams(**params)

In [ ]:
trainer.train()

Epoch 1:


Batch loss = 0.676425: 100%|██████████| 1112/1112 [00:25<00:00, 43.51it/s]



Evaluation (dev):


100%|██████████| 7357/7357 [02:11<00:00, 55.96it/s]


 {'precision': 0.05195052331113225, 'recall': 0.11515703682338883, 'ndcg': 0.10148815950513894}
Epoch 2:


...


Batch loss = 0.674717: 100%|██████████| 1112/1112 [00:23<00:00, 48.21it/s]


Evaluation (dev):



100%|██████████| 7357/7357 [02:12<00:00, 55.63it/s]


 {'precision': 0.05253500067962485, 'recall': 0.11583893124912728, 'ndcg': 0.10218510742709311}


SLRCSimple()

Save users' embeddings for validation step (user's embedding for validation step is his vector of recommendation scores for all items at the time of validation):

In [ ]:
dev_user_emb = trainer.get_predictions(mode="dev")

100%|██████████| 7357/7357 [02:09<00:00, 56.77it/s]


Tune Neighborhood Module hyperparams on validation dataset:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 7358/7358 [00:16<00:00, 456.30it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 4256.65it/s]


test dataset preparing...


100%|██████████| 7357/7357 [00:02<00:00, 3648.96it/s]


In [ ]:
def objective(trial):
    params = {
        "model": NBRKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=1, high=200),
            alpha=trial.suggest_float("alpha", 0.0, 1.0, step=0.05),
            user_emb=dev_user_emb
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-05-02 16:14:51,370] A new study created in memory with name: no-name-78ada8d3-286c-4a92-a7e1-b75ef3db60e8
100%|██████████| 7357/7357 [06:52<00:00, 17.81it/s]
[I 2023-05-02 16:21:52,078] Trial 0 finished with value: 0.09548180430581692 and parameters: {'nearest_neighbors_num': 155, 'alpha': 0.0}. Best is trial 0 with value: 0.09548180430581692.
100%|██████████| 7357/7357 [06:44<00:00, 18.17it/s]
[I 2023-05-02 16:28:42,677] Trial 1 finished with value: 0.10705625742842506 and parameters: {'nearest_neighbors_num': 127, 'alpha': 0.75}. Best is trial 1 with value: 0.10705625742842506.
100%|██████████| 7357/7357 [07:07<00:00, 17.19it/s]
[I 2023-05-02 16:35:56,900] Trial 2 finished with value: 0.11094634153757461 and parameters: {'nearest_neighbors_num': 100, 'alpha': 0.2}. Best is trial 2 with value: 0.11094634153757461.
100%|██████████| 7357/7357 [06:41<00:00, 18.34it/s]
[I 2023-05-02 16:42:45,257] Trial 3 finished with value: 0.10710593535576808 and parameters: {'nearest_neighbors_

Test TAIWI (calculate scores for different seeds):

In [ ]:
test_metrics = {
    "precision": [],
    "recall": [],
    "ndcg": []
}

In [ ]:
for seed in range(5):
    print(f"\n___SEED___{seed}")
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    trainer = NBRTrainer(
        corpus=corpus,
        max_epochs=20,
        topk=10,
        early_stop_num=3
    )

    params = {
        "model": RepurchaseModule(
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": slrc_best_params["batch_size"],
        "lr": slrc_best_params["lr"],
        "l2_reg_coef": slrc_best_params["l2_reg_coef"]
    }

    trainer.init_hyperparams(**params)
    trainer.train()

    dev_user_emb = trainer.get_predictions(mode="dev")
    test_user_emb = trainer.get_predictions(mode="test")

    trainer = NBRTrainer(
        corpus=corpus,
        max_epochs=None,
        topk=10,
        early_stop_num=None
    )

    params = {
        "model": NBRKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
            alpha=study.best_params["alpha"],
            user_emb=dev_user_emb
        )
    }
    params["model"].set_emb(test_user_emb)

    trainer.init_hyperparams(**params)

    metrics = trainer.evaluate(mode="test")

    test_metrics["precision"].append(metrics["precision"])
    test_metrics["recall"].append(metrics["recall"])
    test_metrics["ndcg"].append(metrics["ndcg"])
    print(test_metrics)


___SEED___3
train dataset preparing...


100%|██████████| 7358/7358 [00:11<00:00, 627.64it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 5010.40it/s]


test dataset preparing...


...

dev dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 4578.31it/s]


test dataset preparing...


100%|██████████| 7357/7357 [04:49<00:00, 25.46it/s]

{'precision': [0.06676634497757239, 0.06725567486747315, 0.06739159983689004, 0.06713334239499796, 0.06705178741334783], 'recall': [0.16330173977286047, 0.1645299677770667, 0.16432525138329832, 0.16452850132144814, 0.1645606933393488], 'ndcg': [0.12485593594413333, 0.1264572343117576, 0.1275585838242084, 0.12585491718264302, 0.12593791089122025]}


In [ ]:
{
    "precision": np.array(test_metrics["precision"]).mean(),
    "recall": np.array(test_metrics["recall"]).mean(),
    "ndcg": np.array(test_metrics["ndcg"]).mean(),
}

{'precision': 0.06711974989805627,
 'recall': 0.1642492307188045,
 'ndcg': 0.1261329164307925}

# TaoBao

Fix seed:

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 10 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [ ]:
corpus_path = "./data/"
dataset_name = "taobao"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(10, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 672404, #items = 638962, #clicks = 2015807 (#illegal records = 0)
After preprocessing: #users = 10092, #items = 22286, #clicks = 67991
Saving dataset in ./data//data_taobao/...


In [ ]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Train Repurchase Module with best hyperparams (see ./testing_repurchasemodule.ipynb):

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=20,
    topk=10,
    early_stop_num=3
)

In [ ]:
slrc_best_params = {'batch_size': 256, 'lr': 0.00022155020864083442, 'l2_reg_coef': 0.012687180197268989}

params = {
    "model": RepurchaseModule(
        item_num=corpus.n_items,
        avg_repeat_interval=corpus.total_avg_interval
    ),
    "batch_size": slrc_best_params["batch_size"],
    "lr": slrc_best_params["lr"],
    "l2_reg_coef": slrc_best_params["l2_reg_coef"]
}

trainer.init_hyperparams(**params)

In [ ]:
trainer.train()

Save users' embeddings for validation step (user's embedding for validation step is his vector of recommendation scores for all items at the time of validation):

In [ ]:
dev_user_emb = trainer.get_predictions(mode="dev")

Tune Neighborhood Module hyperparams on validation dataset:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 10092/10092 [00:49<00:00, 205.66it/s]


dev dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 29295.58it/s]


test dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 18293.17it/s]


In [ ]:
def objective(trial):
    params = {
        "model": NBRKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=1, high=200),
            alpha=trial.suggest_float("alpha", 0.0, 1.0, step=0.05),
            user_emb=dev_user_emb
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    trainer.model.reset()
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-05-06 18:57:16,656] A new study created in memory with name: no-name-c8c3fe05-679e-4cfb-b233-e5caa62b533c
100%|██████████| 9307/9307 [28:33<00:00,  5.43it/s]
[I 2023-05-06 19:26:11,499] Trial 0 finished with value: 0.02344135825603466 and parameters: {'nearest_neighbors_num': 155, 'alpha': 0.0}. Best is trial 0 with value: 0.02344135825603466.
100%|██████████| 9307/9307 [29:41<00:00,  5.22it/s]
[I 2023-05-06 19:56:13,777] Trial 1 finished with value: 0.07265987971860889 and parameters: {'nearest_neighbors_num': 127, 'alpha': 0.75}. Best is trial 1 with value: 0.07265987971860889.
100%|██████████| 9307/9307 [29:06<00:00,  5.33it/s]
[I 2023-05-06 20:25:41,039] Trial 2 finished with value: 0.07020255416138836 and parameters: {'nearest_neighbors_num': 100, 'alpha': 0.2}. Best is trial 1 with value: 0.07265987971860889.
100%|██████████| 9307/9307 [28:02<00:00,  5.53it/s]
[I 2023-05-06 20:54:04,652] Trial 3 finished with value: 0.07300336535703099 and parameters: {'nearest_neighbors_

Test TAIWI (calculate scores for different seeds):

In [ ]:
test_metrics = {
    "precision": [],
    "recall": [],
    "ndcg": []
}

In [ ]:
for seed in range(3):
    print(f"\n___SEED___{seed}")
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    trainer = NBRTrainer(
        corpus=corpus,
        max_epochs=20,
        topk=10,
        early_stop_num=3
    )

    params = {
        "model": RepurchaseModule(
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": slrc_best_params["batch_size"],
        "lr": slrc_best_params["lr"],
        "l2_reg_coef": slrc_best_params["l2_reg_coef"]
    }

    trainer.init_hyperparams(**params)
    trainer.train()

    dev_user_emb = trainer.get_predictions(mode="dev")
    test_user_emb = trainer.get_predictions(mode="test")

    trainer = NBRTrainer(
        corpus=corpus,
        max_epochs=None,
        topk=10,
        early_stop_num=None
    )

    params = {
        "model": NBRKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
            alpha=study.best_params["alpha"],
            user_emb=dev_user_emb
        )
    }
    params["model"].set_emb(test_user_emb)

    trainer.init_hyperparams(**params)

    metrics = trainer.evaluate(mode="test")

    trainer.model.reset()

    test_metrics["precision"].append(metrics["precision"])
    test_metrics["recall"].append(metrics["recall"])
    test_metrics["ndcg"].append(metrics["ndcg"])
    print(test_metrics)

In [ ]:
{
    "precision": np.array(test_metrics["precision"]).mean(),
    "recall": np.array(test_metrics["recall"]).mean(),
    "ndcg": np.array(test_metrics["ndcg"]).mean(),
}

{'precision': 0.0123025679596003,
 'recall': 0.11903226961785035,
 'ndcg': 0.0809702514792717}

# Dunnhumby

Fix seed:

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 5 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [ ]:
corpus_path = "./data/"
dataset_name = "dunnhumby"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 2500, #items = 92339, #clicks = 2595370 (#illegal records = 0)
After preprocessing: #users = 2358, #items = 26756, #clicks = 1976796
Saving dataset in ./data//data_dunnhumby/...


In [ ]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Train Repurchase Module with best hyperparams (see ./testing_repurchasemodule.ipynb):

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=20,
    topk=10,
    early_stop_num=3
)

train dataset preparing...


100%|██████████| 2358/2358 [00:13<00:00, 174.26it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:13<00:00, 170.87it/s]


test dataset preparing...


100%|██████████| 2357/2357 [00:13<00:00, 169.82it/s]


In [ ]:
slrc_best_params = {'batch_size': 128, 'lr': 2.0869566476632644e-05, 'l2_reg_coef': 0.0008476179290251597}

params = {
    "model": RepurchaseModule(
        item_num=corpus.n_items,
        avg_repeat_interval=corpus.total_avg_interval
    ),
    "batch_size": slrc_best_params["batch_size"],
    "lr": slrc_best_params["lr"],
    "l2_reg_coef": slrc_best_params["l2_reg_coef"]
}

trainer.init_hyperparams(**params)

In [ ]:
trainer.train()

Epoch 1:


Batch loss = 0.577438: 100%|██████████| 15060/15060 [04:32<00:00, 55.20it/s]


Evaluation (dev):



100%|██████████| 2357/2357 [03:16<00:00, 11.97it/s]


 {'precision': 0.11565549427238016, 'recall': 0.17684104601420847, 'ndcg': 0.1648580594543285}
Epoch 2:


...


Batch loss = 0.571765: 100%|██████████| 15060/15060 [04:12<00:00, 59.57it/s]


Evaluation (dev):



100%|██████████| 2357/2357 [03:15<00:00, 12.05it/s]


 {'precision': 0.11582520152736529, 'recall': 0.17394463845647845, 'ndcg': 0.16410026592749155}


SLRCSimple()

Save users' embeddings for validation step (user's embedding for validation step is his vector of recommendation scores for all items at the time of validation):

In [ ]:
dev_user_emb = trainer.get_predictions(mode="dev")

100%|██████████| 2357/2357 [03:29<00:00, 11.27it/s]


Tune Neighborhood Module hyperparams on validation dataset:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 2358/2358 [00:16<00:00, 145.73it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:12<00:00, 189.84it/s]


test dataset preparing...


100%|██████████| 2357/2357 [00:12<00:00, 184.62it/s]


In [ ]:
def objective(trial):
    params = {
        "model": NBRKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=1, high=200),
            alpha=trial.suggest_float("alpha", 0.0, 1.0, step=0.05),
            user_emb=dev_user_emb
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-05-02 20:44:53,469] A new study created in memory with name: no-name-7325d121-200b-4c25-880c-0966116ae882
100%|██████████| 2357/2357 [03:00<00:00, 13.09it/s]
[I 2023-05-02 20:47:59,098] Trial 0 finished with value: 0.12311387916948324 and parameters: {'nearest_neighbors_num': 155, 'alpha': 0.0}. Best is trial 0 with value: 0.12311387916948324.
100%|██████████| 2357/2357 [02:57<00:00, 13.26it/s]
[I 2023-05-02 20:51:02,569] Trial 1 finished with value: 0.1718346591905977 and parameters: {'nearest_neighbors_num': 127, 'alpha': 0.75}. Best is trial 1 with value: 0.1718346591905977.
100%|██████████| 2357/2357 [02:58<00:00, 13.20it/s]
[I 2023-05-02 20:54:07,413] Trial 2 finished with value: 0.1527396060836034 and parameters: {'nearest_neighbors_num': 100, 'alpha': 0.2}. Best is trial 1 with value: 0.1718346591905977.
100%|██████████| 2357/2357 [02:37<00:00, 14.96it/s]
[I 2023-05-02 20:56:50,192] Trial 3 finished with value: 0.1735124404810969 and parameters: {'nearest_neighbors_num':

Test TAIWI (calculate scores for different seeds):

In [ ]:
test_metrics = {
    "precision": [],
    "recall": [],
    "ndcg": []
}

In [ ]:
for seed in range(5):
    print(f"\n___SEED___{seed}")
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    trainer = NBRTrainer(
        corpus=corpus,
        max_epochs=20,
        topk=10,
        early_stop_num=3
    )

    params = {
        "model": RepurchaseModule(
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": slrc_best_params["batch_size"],
        "lr": slrc_best_params["lr"],
        "l2_reg_coef": slrc_best_params["l2_reg_coef"]
    }

    trainer.init_hyperparams(**params)
    trainer.train()

    dev_user_emb = trainer.get_predictions(mode="dev")
    test_user_emb = trainer.get_predictions(mode="test")

    trainer = NBRTrainer(
        corpus=corpus,
        max_epochs=None,
        topk=10,
        early_stop_num=None
    )

    params = {
        "model": NBRKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
            alpha=study.best_params["alpha"],
            user_emb=dev_user_emb
        )
    }
    params["model"].set_emb(test_user_emb)

    trainer.init_hyperparams(**params)

    metrics = trainer.evaluate(mode="test")

    test_metrics["precision"].append(metrics["precision"])
    test_metrics["recall"].append(metrics["recall"])
    test_metrics["ndcg"].append(metrics["ndcg"])
    print(test_metrics)


___SEED___0
train dataset preparing...


100%|██████████| 2358/2358 [00:08<00:00, 279.27it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:11<00:00, 201.10it/s]


test dataset preparing...


...

dev dataset preparing...


100%|██████████| 2357/2357 [00:10<00:00, 221.07it/s]


test dataset preparing...


100%|██████████| 2357/2357 [01:57<00:00, 20.04it/s]

{'precision': [0.12095884599066611, 0.12134068731438269, 0.12112855324565125, 0.12108612643190496, 0.12104369961815868], 'recall': [0.17612002745219862, 0.17772127477198774, 0.17742432221031992, 0.17745281735556748, 0.17755391459587222], 'ndcg': [0.17098355046152433, 0.17145930304056856, 0.17137393142884408, 0.17143860369949687, 0.17146823846395248]}


In [ ]:
{
    "precision": np.array(test_metrics["precision"]).mean(),
    "recall": np.array(test_metrics["recall"]).mean(),
    "ndcg": np.array(test_metrics["ndcg"]).mean(),
}

{'precision': 0.12111158252015272,
 'recall': 0.1772544712771892,
 'ndcg': 0.17134472541887727}